In [102]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [103]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [104]:
#!wget $data -O data-week-3.csv 

In [105]:

df = pd.read_csv('data-week-3.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [106]:
df.columns = df.columns.str.lower().str.replace(' ','_')
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ','_')

In [107]:
tc = pd.to_numeric(df.totalcharges, errors='coerce')

In [108]:

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')

In [109]:
df.totalcharges = df.totalcharges.fillna(0)

In [110]:
df.churn = (df.churn == 'yes').astype(int)

In [111]:
df[tc.isnull()][['customerid','totalcharges']]

,customerid,totalcharges
488,4472-lvygi,0.0
753,3115-czmzd,0.0
936,5709-lvoeq,0.0
1082,4367-nuyao,0.0
1340,1371-dwpaz,0.0
3331,7644-omvmy,0.0
3826,3213-vvolg,0.0
4380,2520-sgtta,0.0
5218,2923-arzlg,0.0
6670,4075-wkniu,0.0


## Setting upd the validation framework

In [112]:
from sklearn.model_selection import train_test_split

In [113]:
train_test_split?

Signature:
train_test_split(
    *arrays,
    test_size=None,
    train_size=None,
    random_state=None,
    shuffle=True,
    stratify=None,
)
Docstring:
Split arrays or matrices into random train and test subsets.

Quick utility that wraps input validation,
``next(ShuffleSplit().split(X, y))``, and application to input data
into a single call for splitting (and optionally subsampling) data into a
one-liner.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
*arrays : sequence of indexables with same length / shape[0]
    Allowed inputs are lists, numpy arrays, scipy-sparse
    matrices or pandas dataframes.

test_size : float or int, default=None
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    complement of the train size. If ``train_size`` is also None, it will
    be set to 0.25.

trai

In [114]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, train_size = 0.8)

In [115]:
len(df_full_train),len(df_test)

(5634, 1409)

In [116]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [117]:
len(df_train),len(df_val),len(df_test)

(4225, 1409, 1409)

In [118]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [119]:

y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

del df_train['churn']
del df_val['churn']
del df_test['churn']

## 3.4 EDA

In [120]:
df_full_train = df_full_train.reset_index(drop=True)

In [121]:
df_full_train.churn.value_counts(normalize=True)

churn
0    0.735357
1    0.264643
Name: proportion, dtype: float64

In [122]:
global_churn_rate = df_full_train.churn.mean()
round(global_churn_rate, 2)

np.float64(0.26)

In [123]:
df_full_train.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges        float64
churn                 int64
dtype: object

In [124]:
numerical = ['tenure', 'monthlycharges','totalcharges']

In [125]:
df_full_train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [126]:
categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [127]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

## 3.6 - Feature Importance: Churn Rate And Risk Ratio

In [128]:
df_full_train.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,2931-svltv,male,0,yes,yes,40,yes,yes,fiber_optic,yes,...,no,yes,yes,yes,month-to-month,yes,credit_card_(automatic),110.10,4469.10,0
1,6158-dwpzt,male,0,yes,no,9,no,no_phone_service,dsl,no,...,no,no,no,no,one_year,no,bank_transfer_(automatic),24.10,259.80,1
2,6505-oznpg,female,0,no,no,6,no,no_phone_service,dsl,yes,...,yes,yes,yes,yes,two_year,yes,mailed_check,63.40,348.80,0
3,6719-oxybr,male,0,no,no,15,yes,no,fiber_optic,no,...,yes,no,yes,no,month-to-month,no,electronic_check,85.30,1219.85,0
4,0870-vemyl,female,0,no,no,5,yes,no,dsl,yes,...,no,yes,no,no,month-to-month,no,credit_card_(automatic),53.85,259.80,1


In [129]:
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_female

np.float64(0.27039543997149984)

In [130]:
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_male

np.float64(0.25893172974885037)

In [131]:
global_churn = df_full_train.churn.mean()
global_churn

np.float64(0.26464323748668794)

In [132]:
churn_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean()
churn_partner

np.float64(0.19744990892531877)

In [133]:
global_churn - churn_partner

np.float64(0.06719332856136917)

In [134]:
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean()
churn_no_partner

np.float64(0.32848736587054345)

In [135]:
global_churn - churn_no_partner

np.float64(-0.0638441283838555)

# risk ratio

In [136]:
churn_no_partner / global_churn

np.float64(1.2412460223438242)

In [137]:

df_group = df_full_train.groupby('gender').churn.agg(['mean','count'])
df_group['diff'] = df_group['mean'] - global_churn
df_group['risk'] = df_group['mean'] / global_churn
df_group

,mean,count,diff,risk
gender,,,,
female,0.270395,2807,0.005752,1.021736
male,0.258932,2827,-0.005712,0.978418


In [139]:
from Ipython.display import display

ModuleNotFoundError: No module named 'Ipython'

In [140]:
for c in categorical:
    df_group = df_full_train.groupby(c).churn.agg(['mean','count'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)
    print()
    print()

,mean,count,diff,risk
gender,,,,
female,0.270395,2807,0.005752,1.021736
male,0.258932,2827,-0.005712,0.978418


,mean,count,diff,risk
seniorcitizen,,,,
0,0.233157,4735,-0.031486,0.881025
1,0.430478,899,0.165835,1.626636


,mean,count,diff,risk
partner,,,,
no,0.328487,2889,0.063844,1.241246
yes,0.197450,2745,-0.067193,0.746098


,mean,count,diff,risk
dependents,,,,
no,0.312215,3946,0.047572,1.179758
yes,0.153436,1688,-0.111207,0.579784


,mean,count,diff,risk
phoneservice,,,,
no,0.245009,551,-0.019634,0.925809
yes,0.266772,5083,0.002128,1.008042


...

,mean,count,diff,risk
streamingtv,,,,
no,0.331087,2226,0.066444,1.251070
no_internet_service,0.069672,1220,-0.194971,0.263268
yes,0.305759,2188,0.041115,1.155362


,mean,count,diff,risk
streamingmovies,,,,
no,0.329406,2207,0.064763,1.244719
no_internet_service,0.069672,1220,-0.194971,0.263268
yes,0.307657,2207,0.043014,1.162537


,mean,count,diff,risk
contract,,,,
month-to-month,0.426901,3078,0.162257,1.613117
one_year,0.114644,1195,-0.149999,0.433203
two_year,0.029390,1361,-0.235253,0.111056


,mean,count,diff,risk
paperlessbilling,,,,
no,0.155479,2309,-0.109165,0.587502
yes,0.340451,3325,0.075808,1.286453


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.169707,1261,-0.094937,0.641266
credit_card_(automatic),0.154286,1225,-0.110358,0.582995
electronic_check,0.455032,1868,0.190389,1.719417
mailed_check,0.185938,1280,-0.078706,0.702597


## 3.6 - Feature Importance: Mutual Information

In [ ]:

from sklearn.metrics import mutual_info_score

In [142]:
mutual_info_score(df_full_train.churn, df_full_train.contract)

0.09746309965707407

In [143]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

In [144]:
mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending = False)

contract            0.097463
techsupport         0.065421
onlinesecurity      0.063581
internetservice     0.058649
onlinebackup        0.047416
deviceprotection    0.045446
paymentmethod       0.044824
streamingtv         0.033153
streamingmovies     0.033077
paperlessbilling    0.022276
dependents          0.014593
seniorcitizen       0.012393
partner             0.011142
multiplelines       0.001003
phoneservice        0.000109
gender              0.000084
dtype: float64

## 3.7 - Feature Importance: Correlation

In [146]:
df_full_train[numerical].corrwith(df_full_train.churn)

tenure           -0.346539
monthlycharges    0.205257
totalcharges     -0.190826
dtype: float64

In [148]:
df_full_train[df_full_train.tenure > 12].churn.mean()

np.float64(0.17428131416837783)